# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
#pip install gmaps

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [3]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [4]:
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [5]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Belmonte,BR,-15.8631,-38.8828,72.28,90,92,1.88,overcast clouds
1,1,Udachnyy,RU,66.4167,112.4000,-8.50,98,100,2.95,overcast clouds
2,2,Thompson,CA,55.7435,-97.8558,24.96,100,100,0.00,light snow
3,3,Tuatapere,NZ,-46.1333,167.6833,58.60,52,94,7.76,overcast clouds
4,4,Cherskiy,RU,68.7500,161.3000,-38.42,100,32,1.30,scattered clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [6]:
# Prompt the user to enter the minimum temperature criteria
min_temp =  float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [7]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Belmonte,BR,-15.8631,-38.8828,72.28,90,92,1.88,overcast clouds
5,5,Atuona,PF,-9.8000,-139.0333,78.51,72,36,17.40,scattered clouds
8,8,Salalah,OM,17.0151,54.0924,78.89,47,20,2.30,few clouds
10,10,Hilo,US,19.7297,-155.0900,73.13,78,100,8.05,overcast clouds
16,16,Mumford,GH,5.2618,-0.7590,78.40,81,9,2.93,clear sky
17,17,Wawa,NG,9.9022,4.4192,70.12,32,2,3.02,clear sky
18,18,East London,ZA,-33.0153,27.9116,67.14,88,0,5.75,clear sky
23,23,Carutapera,BR,-1.1950,-46.0200,75.99,91,9,5.28,clear sky
25,25,Busselton,AU,-33.6500,115.3333,76.21,50,28,9.95,scattered clouds
26,26,Port Elizabeth,ZA,-33.9180,25.5701,69.31,91,0,5.75,clear sky


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [8]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities_df = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Belmonte,BR,-15.8631,-38.8828,72.28,90,92,1.88,overcast clouds
5,5,Atuona,PF,-9.8000,-139.0333,78.51,72,36,17.40,scattered clouds
8,8,Salalah,OM,17.0151,54.0924,78.89,47,20,2.30,few clouds
10,10,Hilo,US,19.7297,-155.0900,73.13,78,100,8.05,overcast clouds
16,16,Mumford,GH,5.2618,-0.7590,78.40,81,9,2.93,clear sky
...,...,...,...,...,...,...,...,...,...,...
546,546,Gizo,SB,-8.1030,156.8419,85.71,64,22,5.59,few clouds
548,548,Bandarbeyla,SO,9.4942,50.8122,78.87,69,52,15.93,broken clouds
552,552,Bambanglipuro,ID,-7.9500,110.2833,83.25,70,100,5.99,overcast clouds
553,553,Inirida,CO,3.8653,-67.9239,73.65,95,67,3.27,broken clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [9]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Weather Description"]].copy()

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Weather Description
0,Belmonte,BR,72.28,-15.8631,-38.8828,overcast clouds
5,Atuona,PF,78.51,-9.8000,-139.0333,scattered clouds
8,Salalah,OM,78.89,17.0151,54.0924,few clouds
10,Hilo,US,73.13,19.7297,-155.0900,overcast clouds
16,Mumford,GH,78.40,5.2618,-0.7590,clear sky
17,Wawa,NG,70.12,9.9022,4.4192,clear sky
18,East London,ZA,67.14,-33.0153,27.9116,clear sky
23,Carutapera,BR,75.99,-1.1950,-46.0200,clear sky
25,Busselton,AU,76.21,-33.6500,115.3333,scattered clouds
26,Port Elizabeth,ZA,69.31,-33.9180,25.5701,clear sky


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [10]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Weather Description,Hotel Name
0,Belmonte,BR,72.28,-15.8631,-38.8828,overcast clouds,
5,Atuona,PF,78.51,-9.8000,-139.0333,scattered clouds,
8,Salalah,OM,78.89,17.0151,54.0924,few clouds,
10,Hilo,US,73.13,19.7297,-155.0900,overcast clouds,
16,Mumford,GH,78.40,5.2618,-0.7590,clear sky,
17,Wawa,NG,70.12,9.9022,4.4192,clear sky,
18,East London,ZA,67.14,-33.0153,27.9116,clear sky,
23,Carutapera,BR,75.99,-1.1950,-46.0200,clear sky,
25,Busselton,AU,76.21,-33.6500,115.3333,scattered clouds,
26,Port Elizabeth,ZA,69.31,-33.9180,25.5701,clear sky,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [11]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [12]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Weather Description,Hotel Name
0,Belmonte,BR,72.28,-15.8631,-38.8828,overcast clouds,Pousada Monte Carmelo O Pão
5,Atuona,PF,78.51,-9.8000,-139.0333,scattered clouds,Villa Enata
8,Salalah,OM,78.89,17.0151,54.0924,few clouds,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
10,Hilo,US,73.13,19.7297,-155.0900,overcast clouds,Hilo Hawaiian Hotel
16,Mumford,GH,78.40,5.2618,-0.7590,clear sky,Peace Guest House


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [14]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = clean_hotel_df = hotel_df.dropna(axis="index", how="any")

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Weather Description,Hotel Name
0,Belmonte,BR,72.28,-15.8631,-38.8828,overcast clouds,Pousada Monte Carmelo O Pão
5,Atuona,PF,78.51,-9.8000,-139.0333,scattered clouds,Villa Enata
8,Salalah,OM,78.89,17.0151,54.0924,few clouds,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
10,Hilo,US,73.13,19.7297,-155.0900,overcast clouds,Hilo Hawaiian Hotel
16,Mumford,GH,78.40,5.2618,-0.7590,clear sky,Peace Guest House
17,Wawa,NG,70.12,9.9022,4.4192,clear sky,
18,East London,ZA,67.14,-33.0153,27.9116,clear sky,Tu Casa
23,Carutapera,BR,75.99,-1.1950,-46.0200,clear sky,Lidera
25,Busselton,AU,76.21,-33.6500,115.3333,scattered clouds,Observatory Guest House
26,Port Elizabeth,ZA,69.31,-33.9180,25.5701,clear sky,39 On Nile Guest House


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [15]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [16]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label='City_ID')

##### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [17]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt> Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [18]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Pousada Monte Carmelo O Pão</dd>\n<dt>City</dt><dd>Belmonte</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt> Weather</dt><dd>overcast clouds and 72.28 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Villa Enata</dd>\n<dt>City</dt><dd>Atuona</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt> Weather</dt><dd>scattered clouds and 78.51 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel</dd>\n<dt>City</dt><dd>Salalah</dd>\n<dt>Country</dt><dd>OM</dd>\n<dt> Weather</dt><dd>few clouds and 78.89 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hilo Hawaiian Hotel</dd>\n<dt>City</dt><dd>Hilo</dd>\n<dt>Country</dt><dd>US</dd>\n<dt> Weather</dt><dd>overcast clouds and 73.13 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Peace Guest House</dd>\n<dt>City</dt><dd>Mumford</dd>\n<dt>Country</dt><dd>GH</dd>\n<dt> Weather</dt><dd>clear sky and 78.4 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Wawa</dd>\n<d

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [19]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
0,-15.8631,-38.8828
5,-9.8000,-139.0333
8,17.0151,54.0924
10,19.7297,-155.0900
16,5.2618,-0.7590
17,9.9022,4.4192
18,-33.0153,27.9116
23,-1.1950,-46.0200
25,-33.6500,115.3333
26,-33.9180,25.5701


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [20]:
# Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Create a figure to add the Google map as a layer
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))